In [3]:
from pathlib import Path
import torch
import pandas as pd
from forest_fire_snn import SNN


BASE_DIR = Path.cwd()

net = SNN()
model_path = BASE_DIR / "trained_snn_model.pth"
net.load_state_dict(torch.load(model_path, map_location="cpu"))
net.eval()
print("Trained model loaded successfully.")

csv_path = BASE_DIR / "firedata_validation.csv"
df = pd.read_csv(csv_path)

# normalize once
df["Temp"] = df["Temp"] / 100
df["Humidity"] = df["Humidity"] / 100
df["CO2"] = df["CO2"] / 5000

num_steps = 25

preds, confs, score0, score1 = [], [], [], []

with torch.no_grad():
    for _, row in df.iterrows():
        x0 = torch.tensor(
            [[row["Temp"], row["Audio"], row["Humidity"], row["CO2"]]],
            dtype=torch.float32
        )
        x = x0.unsqueeze(0).repeat(num_steps, 1, 1)

        spk_rec, mem_rec = net(x)
        spike_counts = spk_rec.sum(dim=0)

        s0 = float(spike_counts[0, 0].item())
        s1 = float(spike_counts[0, 1].item())
        pred = int(torch.argmax(spike_counts, dim=1).item())
        conf = float(spike_counts[0, pred].item() / (spike_counts[0].sum().item() + 1e-9))

        score0.append(s0); score1.append(s1)
        preds.append(pred); confs.append(conf)

df["Spikes_Class0"] = score0
df["Spikes_Class1"] = score1
df["Pred_Fire"] = preds
df["Pred_Conf"] = confs

out_path = BASE_DIR / "firedata_validation_with_preds.csv"
df.to_csv(out_path, index=False)
print("Saved:", out_path)
print(df.head())

Trained model loaded successfully.
Saved: c:\Users\Aishik C\Desktop\neuromorphic\neuromorphic_project\firedata_validation_with_preds.csv
       Temp     Audio  Humidity       CO2  Fire  Spikes_Class0  Spikes_Class1  \
0  0.495897  0.392037  0.268400  0.379641     1            8.0            8.0   
1  0.304385 -0.068695  0.259834  0.213566     0            8.0            7.0   
2  0.307251  0.761692  0.181879  0.207472     0           11.0            8.0   
3  0.176588  0.591195  0.237749  0.481001     0           12.0            7.0   
4  0.100885  0.617177  0.147729  0.047089     0           12.0            7.0   

   Pred_Fire  Pred_Conf  
0          0   0.500000  
1          0   0.533333  
2          0   0.578947  
3          0   0.631579  
4          0   0.631579  


In [4]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_true = df["Fire"].astype(int).to_numpy()
y_pred = df["Pred_Fire"].astype(int).to_numpy()

print("Accuracy:", accuracy_score(y_true, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, digits=4))

Accuracy: 0.9091
Confusion matrix:
 [[14919    40]
 [ 2687 12354]]
              precision    recall  f1-score   support

           0     0.8474    0.9973    0.9163     14959
           1     0.9968    0.8214    0.9006     15041

    accuracy                         0.9091     30000
   macro avg     0.9221    0.9093    0.9084     30000
weighted avg     0.9223    0.9091    0.9084     30000



In [5]:
# sanity check balance
print(df["Fire"].value_counts())
print(df["Fire"].value_counts(normalize=True))

# accuracy + confusion breakdown (no sklearn needed)
y_true = df["Fire"].astype(int).to_numpy()
y_pred = df["Pred_Fire"].astype(int).to_numpy()

tp = ((y_true==1) & (y_pred==1)).sum()
tn = ((y_true==0) & (y_pred==0)).sum()
fp = ((y_true==0) & (y_pred==1)).sum()
fn = ((y_true==1) & (y_pred==0)).sum()

acc = (tp+tn)/len(y_true)
precision = tp/(tp+fp+1e-9)
recall = tp/(tp+fn+1e-9)
f1 = 2*precision*recall/(precision+recall+1e-9)

print(f"Accuracy:  {acc:.4f}")
print(f"TP={tp}  TN={tn}  FP={fp}  FN={fn}")
print(f"Precision: {precision:.4f}  (how often 'fire' is correct)")
print(f"Recall:    {recall:.4f}  (how many fires you catch)")
print(f"F1:        {f1:.4f}")


Fire
1    15041
0    14959
Name: count, dtype: int64
Fire
1    0.501367
0    0.498633
Name: proportion, dtype: float64
Accuracy:  0.9091
TP=12354  TN=14919  FP=40  FN=2687
Precision: 0.9968  (how often 'fire' is correct)
Recall:    0.8214  (how many fires you catch)
F1:        0.9006
